# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы. Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [216]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.dummy import DummyClassifier

In [4]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Построим модель бинарной классификации целевого признака 'is_ultra' по 4 признакам

## Разбейте данные на выборки

Разметим данные на признаки features  и целевой признак target

In [14]:
features = df.drop('is_ultra', axis=1)
target = df['is_ultra']
n = df.shape[0]

3214

Разобъем выборку на тренировочную и тестовую

In [122]:
X_train, X_test, y_train, y_test = train_test_split(
    features,
    target,
    random_state=543,
    test_size=0.2
)

Теперь из тренировночной выделим валидационную выборку

In [209]:
features_train, features_valid, target_train, target_valid = train_test_split(
    X_train,
    y_train,
    test_size=0.25,
    random_state=17
)

Проверим объемы выборок (60%, 20%, 20%)

In [124]:
print(f'Размер обучающего набора: {features_train.shape[0]/n}\
        размер проверочного набора: {features_valid.shape[0]/n}\
        размер тестового набора: {X_test.shape[0]/n}'\
     )

Размер обучающего набора: 0.5998755444928439        размер проверочного набора: 0.2000622277535781        размер тестового набора: 0.2000622277535781


Проверим среднее целевых признаков выборок, чтобы оно не отличалось сильно те было репрезентативно, зафиксируем random_state

In [210]:
print(f'Среднее обучающего набора: {target_train.mean()}\
        среднее проверочного набора: {target_valid.mean()}\
        среднее тестового набора: {y_test.mean()}'\
     )

Среднее обучающего набора: 0.3070539419087137        среднее проверочного набора: 0.3094867807153966        среднее тестового набора: 0.30171073094867806


## Исследуйте модели

Нам предстоит выбрать модель бинарной классификации

Первым проверим дерево решений

In [136]:
tree = DecisionTreeClassifier(
    random_state=17,
    max_depth=5
)

In [137]:
tree.fit(features_train, target_train)
tree_pred = tree.predict(features_valid)
accuracy_score(target_valid, tree_pred)

0.7776049766718507

С помощью GridSearchCV подберем оптимальные гиперпараметры

In [149]:
tree_params = {'max_depth': list(range(1,100))}

Для каждого значения параметра max_depth (максимальная глубина дерева) будет проведена 5-кратная кросс-валидация (сv) и выберется лучшее сочетание параметров

In [211]:
tree_grid = GridSearchCV(tree,
                         tree_params,
                         cv=5,
                         n_jobs=-1
                        )

In [212]:
tree_grid.fit(features_train, target_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=5,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=17,
                                              splitter='best'),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                       13, 14, 15, 16, 17, 18, 19

In [152]:
print(tree_grid.best_params_, tree_grid.best_score_)

{'max_depth': 10} 0.8008298755186722


Лучшая модель при max_depth = 10

Посчитаем accuracy_score с таким значением гипермапарметра

In [153]:
accuracy_score(target_valid, tree_grid.predict(features_valid))

0.7791601866251944

Следующая модель - Случайный лес

In [156]:
forest = RandomForestClassifier(
    random_state=17,
    max_depth=10,
    n_estimators=5
)

In [157]:
forest.fit(features_train, target_train)
forest_pred = forest.predict(features_valid)
accuracy_score(target_valid, forest_pred)

0.7744945567651633

В этой модели добавим гиперпараметр характерный для ансамбля - n_estimators (число деревьев в "лесу")

In [158]:
forest_params = {'max_depth': list(range(1,20)),
                'n_estimators': list(range(1,10))
                }

In [159]:
forest_grid = GridSearchCV(forest,
                           forest_params,
                           cv=5,
                           n_jobs=-1
                          )

In [160]:
forest_grid.fit(features_train, target_train)

Fitting 5 folds for each of 891 candidates, totalling 4455 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-1)]: Done 4455 out of 4455 | elapsed:  2.0min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=10,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=5, n_jobs=None,
                                              oob_score=False, random_state=17,
                                              verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'max_depth

In [162]:
print(forest_grid.best_params_, forest_grid.best_score_)

{'max_depth': 9, 'n_estimators': 8} 0.8106846473029046


Лучшие значения - max_depth=9, n_estimators=8

In [163]:
accuracy_score(target_valid, forest_grid.predict(features_valid))

0.7776049766718507

accuracy на валидационной выборке - 0.7776, хуже чем у дерева решений

Посмотрим следующую модель классификации - логистическую регрессию с гиперпараметром С (обратного коэффициента регуляризации)

In [170]:
logit = LogisticRegression(
    C=1,
    random_state=17
)

In [171]:
logit.fit(features_train, target_train)
logit_pred = logit.predict(features_valid)
accuracy_score(target_valid, logit_pred)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.7013996889580093

In [217]:
logit_params = {'C': list(range(1,1000))}

In [218]:
logit_grid = GridSearchCV(logit,
                          logit_params,
                          cv=5,
                          n_jobs=-1
                         )

In [219]:
logit_grid.fit(features_train, target_train)

Fitting 5 folds for each of 1000 candidates, totalling 5000 fits


[Parallel(n_jobs=-1)]: Using backend SequentialBackend with 1 concurrent workers.
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver wil

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=17, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'C': [0.001, 1.001, 2.001, 3.0...
                               15.000999999999998, 16.000999999999998,
                               17.000999999999998, 18.000999999999998,
                               19.000999999999998, 20.000999999999998,
                               21.000999999999998, 22.000999999999998,
      

In [220]:
print(logit_grid.best_params_, logit_grid.best_score_)

{'C': 874.0009999999999} 0.7297717842323651


In [176]:
accuracy_score(target_valid, logit_grid.predict(features_valid))

0.7013996889580093

accuracy получилась ниже всех рассмотренных моделей, лучшая у дерева решений

## Проверьте модель на тестовой выборке

In [177]:
accuracy_score(y_test, tree_grid.predict(X_test))

0.7931570762052877

accuracy на тестовой выборке - 0.7931570762052877

## (бонус) Проверьте модели на адекватность

Проверим баланс классов

In [192]:
print(f'Баланс классов обучающего набора:\n {target_train.value_counts()} \n\
        баланс классов проверочного набора:\n {target_valid.value_counts()[0:2]} \n\
        баланс классов тестового набора:\n {y_test.value_counts()[0:2]}'\
     )

Баланс классов обучающего набора:
 0    1336
1     592
Name: is_ultra, dtype: int64 
        баланс классов проверочного набора:
 0    444
1    199
Name: is_ultra, dtype: int64 
        баланс классов тестового набора:
 0    449
1    194
Name: is_ultra, dtype: int64


In [193]:
print(f'Баланс классов всей выборки:\n {target.value_counts()}')

Баланс классов всей выборки:
 0    2229
1     985
Name: is_ultra, dtype: int64


Наблюдаем дисбаланс классов. Делать оценку модели на основе метрики accuracy не совсем корректно

In [ ]:
Построим confusion matrix

In [205]:
y_true = target_valid
y_pred = tree_pred

In [206]:
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
tn, fp, fn, tp

(408, 36, 107, 92)

Чаще всего модель, ожидаемо, ошибается на False Negative (107)
Наверное, при дисбалансе классов, одной из релевантных метрик будет ROC AUC

In [ ]:
Посмотрим какую точность дает Dummy классификатор.
Первое значение accuracy для стратегии предсказания самых часто встречаемях классов в выборке

In [199]:
dummy = DummyClassifier(strategy="most_frequent")
dummy.fit(features_train, target_train)
dummy_pred = dummy.predict(features_valid)
accuracy_score(target_valid, dummy_pred)

0.6905132192846034

In [ ]:
Второе значение accuracy для стратегии случайных предсказаний

In [198]:
dummy = DummyClassifier(strategy="uniform")
dummy.fit(features_train, target_train)
dummy_pred = dummy.predict(features_valid)
accuracy_score(target_valid, dummy_pred)

0.48367029548989116

Оценка полученная на TreeClassifier адекватна тк отличается от классификатора, который делает прогнозы используя простые правила.
Для оценки модели лучше использовать более сложные метрики подходящие под бизнес задачи